In [3]:
# basics
import numpy as np
import pandas as pd
import datetime
import re
import os,glob
import pickle
import textwrap

# statistics
import statsmodels.api as sm
import statsmodels.formula.api as smf


# plotting
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [15,12]

# own data wrappers
from imp import reload
import measureclass as mc; reload(mc);
import coronadataclass as cdc; reload(cdc);
import crossvalidationclass as cvc; reload(cvc);

np.seterr(divide = 'ignore');


In [4]:
cccsl     = mc.COVID19_measures(datasource = 'CCCSL',     download_data = True)
oxford    = mc.COVID19_measures(datasource = 'OXFORD',    download_data = True)
whophsm   = mc.COVID19_measures(datasource = 'WHOPHSM',   download_data = True)
acaps     = mc.COVID19_measures(datasource = 'ACAPS',     download_data = True)
coronanet = mc.COVID19_measures(datasource = 'CORONANET', download_data = True)
hitcovid  = mc.COVID19_measures(datasource = 'HITCOVID',  download_data = True)

In [5]:
print('           {:8s} {:9s}'.format('Measures','Countries'))
print('CCCSL:     {:8d} {:9d}'.format(cccsl.MeasureList(mincount = 5).shape[0],     len(cccsl.countrylist)))
print('OXFORD:    {:8d} {:9d}'.format(oxford.MeasureList(mincount = 5).shape[0],    len(oxford.countrylist)))
print('WHOPHSM:   {:8d} {:9d}'.format(whophsm.MeasureList(mincount = 5).shape[0],   len(whophsm.countrylist)))
print('ACAPS:     {:8d} {:9d}'.format(acaps.MeasureList(mincount = 5).shape[0],     len(acaps.countrylist)))
print('CORONANET: {:8d} {:9d}'.format(coronanet.MeasureList(mincount = 5).shape[0], len(coronanet.countrylist)))
print('HITCOVID:  {:8d} {:9d}'.format(hitcovid.MeasureList(mincount = 5).shape[0],  len(hitcovid.countrylist)))


           Measures Countries
CCCSL:           55        57
OXFORD:          17       184
WHOPHSM:         41       213
ACAPS:           34       194
CORONANET:      109       198
HITCOVID:        39       136


In [6]:
print('           {:8s} {:9s}'.format('Measures','Countries'))
print('CCCSL:     {:8d} {:9d}'.format(cccsl.MeasureList().shape[0],     len(cccsl.countrylist)))
print('OXFORD:    {:8d} {:9d}'.format(oxford.MeasureList().shape[0],    len(oxford.countrylist)))
print('WHOPHSM:   {:8d} {:9d}'.format(whophsm.MeasureList().shape[0],   len(whophsm.countrylist)))
print('ACAPS:     {:8d} {:9d}'.format(acaps.MeasureList().shape[0],     len(acaps.countrylist)))
print('CORONANET: {:8d} {:9d}'.format(coronanet.MeasureList().shape[0], len(coronanet.countrylist)))
print('HITCOVID:  {:8d} {:9d}'.format(hitcovid.MeasureList().shape[0],  len(hitcovid.countrylist)))


           Measures Countries
CCCSL:           66        57
OXFORD:          17       184
WHOPHSM:         45       213
ACAPS:           46       194
CORONANET:      141       198
HITCOVID:        39       136


In [7]:
# generate a full table for a database

def AddDF(df1,df2):
    if not df1 is None:
        if df2 is None:
            return df1
        else:
            return pd.concat([df1,df2], ignore_index = True, sort = False)
    else:
        return df2
    

finaltable = None
for country in cccsl.countrylist:
    country_implementation_table = cccsl.ImplementationTable(country = country, mincount = 5).reset_index().rename(columns = {'index':'Date'})
    country_implementation_table['country'] = country
    finaltable = AddDF(finaltable, country_implementation_table)
finaltable.fillna(0, inplace = True)

finaltable

,Date,AdaptProceduresForPatientManagement,AirportHealthCheck,AirportRestriction,BorderRestriction,ClosureOfEducationalInstitutions,CrisisManagementPlans,EducateAndActivelyCommunicateWithThePublic,EnhanceDetectionSystem,EnhanceLaboratoryTestingCapacity,...,ReceiveInternationalHelp,PortAndShipRestriction,RepurposeHospitals,Research,Surveillance,MeasuresForPublicTransport,TheGovernmentProvideAssistanceToVulnerablePopulations,WorkSafetyProtocols,AccessToNonEssentialcriticalHealthcareServices,ExemptionOfQuarantine
0,22/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,23/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,24/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,25/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,26/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11509,06/08/2020,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
11510,07/08/2020,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
11511,08/08/2020,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
11512,09/08/2020,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0


In [8]:
external_cccsl = pd.read_csv('COVID19_data_PAPER_VERSION.csv', sep = ';')
external_whophsm = pd.read_csv('COVID19_data_PAPER_VERSION_WHOPHSM.csv', sep = ';')
external_coronanet = pd.read_csv('COVID19_data_PAPER_VERSION_CORONANET.csv', sep = ';')
external_oxford = pd.read_csv('COVID19_data_PAPER_VERSION_OXFORD.csv', sep = ';')

In [9]:
oxford_ext_cols = list(external_oxford.columns)
for measureID in oxford.MeasureList(mincount=5).index:
    if measureID in oxford_ext_cols:
        print('found:      {}'.format(measureID))
        oxford_ext_cols.remove(measureID)
    else:
        print('not found:  {}'.format(measureID))
print('remaining: ', oxford_ext_cols)
print('difference:', set(oxford.MeasureList().index).difference(set(oxford.MeasureList(mincount=5).index)))

found:      C1SchoolClosing
found:      C2WorkplaceClosing
found:      C3CancelPublicEvents
found:      C4RestrictionsOnGatherings
found:      C5ClosePublicTransport
found:      C6StayAtHomeRequirements
found:      C7RestrictionsOnInternalMovement
found:      C8InternationalTravelControls
found:      E1IncomeSupport
found:      E2DebtcontractRelief
found:      E3FiscalMeasures
found:      E4InternationalSupport
found:      H1PublicInformationCampaigns
found:      H2TestingPolicy
found:      H3ContactTracing
found:      H4EmergencyInvestmentInHealthcare
found:      H5InvestmentInVaccines
remaining:  ['Country', 'Date', 'Population [million]', 'Population Density', 'GDP pc PPP', 'HDI', 'VA', 'PS', 'GE', 'RQ', 'RL', 'CC', 'Confirmed', 'Deaths', 'Recovered', 'Growth rate', 'R']
difference: set()


In [10]:
cccsl_ext_cols = list(external_cccsl.columns)
for measureID,measuredata in cccsl.MeasureList(mincount=5).iterrows():
    mID = measuredata['Measure_L1'] + ' -- ' + measuredata['Measure_L2']
    if mID in cccsl_ext_cols:
        print('found:      {}'.format(mID))
        cccsl_ext_cols.remove(mID)
    else:
        print('not found:  {}'.format(mID))
print('remaining: ', cccsl_ext_cols)
print('difference:', set(cccsl.MeasureList().index).difference(set(cccsl.MeasureList(mincount=5).index)))

found:      Case identification, contact tracing and related measures -- Activate case notification
found:      Case identification, contact tracing and related measures -- Airport health check
found:      Case identification, contact tracing and related measures -- Border health check
found:      Case identification, contact tracing and related measures -- Enhance detection system
found:      Case identification, contact tracing and related measures -- Isolation of cases
found:      Case identification, contact tracing and related measures -- Quarantine
found:      Case identification, contact tracing and related measures -- Restricted testing
found:      Case identification, contact tracing and related measures -- Surveillance
found:      Case identification, contact tracing and related measures -- Tracing and tracking
found:      Environmental measures -- Environmental cleaning and disinfection
found:      Healthcare and public health capacity -- Adapt procedures for patient managem

In [11]:
whophsm_ext_cols = list(external_whophsm.columns)
for measureID in whophsm.MeasureList(mincount = 5).index:
    if measureID in whophsm_ext_cols:
        print('found:      {}'.format(measureID))
        whophsm_ext_cols.remove(measureID)
    else:
        print('not found:  {}'.format(measureID))
print('remaining: ', whophsm_ext_cols)
print('difference:', set(whophsm.MeasureList().index).difference(set(whophsm.MeasureList(mincount=5).index)))

found:      CleaningAndDisinfectingSurfacesAndObjects
found:      PerformingHandHygiene
found:      PhysicalDistancing
found:      UsingOtherPersonalProtectiveEquipment
found:      WearingAMask
found:      ClosingInternationalLandBorders
found:      EntryScreeningAndIsolationOrQuarantine
found:      ExitScreeningAndIsolationOrQuarantine
found:      ProvidingTravelAdviceOrWarning
found:      RestrictingEntry
found:      RestrictingExit
found:      RestrictingVisas
found:      SuspendingOrRestrictingInternationalFerriesOrShips
found:      SuspendingOrRestrictingInternationalFlights
found:      CommunicationsAndEngagement
found:      CommunicationsAndEngagementGeneralPublicAwarenessCampaigns
found:      CommunicationsAndEngagementOtherCommunications
found:      FinancialPackages
found:      LegalAndPolicyRegulations
found:      Other
found:      ScalingUp
found:      DomesticTravelClosingInternalLandBorders
found:      DomesticTravelRestrictingEntry
found:      DomesticTravelStayAtHomeOrd

In [12]:
coronanet_ext_cols = list(external_coronanet.columns)
for measureID in coronanet.MeasureList(mincount = 5).index:
    if measureID in coronanet_ext_cols:
        print('found:      {}'.format(measureID))
        coronanet_ext_cols.remove(measureID)
    else:
        print('not found:  {}'.format(measureID))
print('remaining: ', coronanet_ext_cols)
print('difference:', set(coronanet.MeasureList().index).difference(set(coronanet.MeasureList(mincount=5).index)))

found:      AntiDisinformationMeasures
found:      NoSpecialPopulationTargeted
found:      ClosureAndRegulationOfSchools
not found:  PreschoolOrChildcareFacilitiesGenerallyForChildrenAges5AndBelow
not found:  PrimarySchoolsGenerallyForChildrenAges10AndBelow
not found:  SecondarySchoolsGenerallyForChildrenAges10To18
found:      Curfew
found:      DeclarationOfEmergency
found:      ExternalBorderRestrictions
found:      HealthCertificates
not found:  HealthScreeningsEgTemperatureChecks
found:      OtherExternalBorderRestriction
not found:  TravelHistoryFormEgDocumentsWhereTravelerHasRecentlyBeen
not found:  VisaExtensionsEgVisaValidityExtended
not found:  VisaRestrictionsEgSuspendIssuanceOfVisa
found:      HealthMonitoring
found:      Doctors
found:      HandSanitizer
found:      HealthInsurance
found:      HealthResearchFacilities
found:      HealthResources
found:      HealthVolunteers
found:      Hospitals
found:      Masks
found:      Medicinedrugs
found:      Nurses
found:      Othe